La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

Lectura del CSV

In [2]:
steam_games = pd.read_csv('..\\Datsets\\CSV\\steam_games.csv')
user_items = pd.read_csv('..\\Datsets\\CSV\\user_items.csv')
user_reviews = pd.read_csv('..\\Datsets\\CSV\\user_reviews.csv')

Colimas que necesito : user_id , items_count , item_id , item_name , playtime_forever y recommend
Hacemos un left joins porque no todos los usuarios , han hecho recomendaciones


In [5]:
df_user_item = pd.merge(user_items, user_reviews[['user_id', 'recommend']], on='user_id', how='left')

# Seleccionar las columnas deseadas
df_user_item = df_user_item[['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever', 'recommend']]

In [6]:
df_user_item

,user_id,items_count,item_id,item_name,playtime_forever,recommend
0,76561197970982479,277,10,Counter-Strike,6,True
1,76561197970982479,277,10,Counter-Strike,6,True
2,76561197970982479,277,10,Counter-Strike,6,True
3,76561197970982479,277,20,Team Fortress Classic,0,True
4,76561197970982479,277,20,Team Fortress Classic,0,True
...,...,...,...,...,...,...
9965094,76561198329548331,7,346330,BrainBread 2,0,NaN
9965095,76561198329548331,7,373330,All Is Dust,0,NaN
9965096,76561198329548331,7,388490,One Way To Die: Steam Edition,3,NaN
9965097,76561198329548331,7,521570,You Have 10 Seconds 2,4,NaN


In [7]:
# Agrupar por user_id, item_name y mantener las columnas necesarias
df_user_item = df_user_item.groupby(['user_id', 'item_name']).agg({
    'items_count': 'first',  # Puedes usar 'first' para mantener un valor único
    'playtime_forever': 'sum',
    'recommend': 'first'  # Similar a 'items_count', selecciona el primer valor
}).reset_index()

## Preprosesamiento de los datos

Para saber que usuarios son parecidos con las columnas que tenemos son las columnas de items_count y playtime_forever

In [8]:
# Agrupar por user_id
df_user_item_ML = df_user_item.groupby(['user_id']).agg({
    'items_count': 'first',  # Puedes usar 'first' para mantener un valor único
    'playtime_forever': 'sum',
}).reset_index()

Por el EDA sabemos que las columnas items_counts y playtime_forever estan sesgadas a la derecha por lo que se aplicada una transformacion logaritmica

In [9]:
# Aplicar transformación logarítmica a las columnas
df_user_item_ML['items_count'] = np.log1p(df_user_item_ML['items_count'])
df_user_item_ML['playtime_forever'] = np.log1p(df_user_item_ML['playtime_forever'])

In [10]:
df_user_item_ML.columns

Index(['user_id', 'items_count', 'playtime_forever'], dtype='object')

Creamos un dataframe con dos columnas una del id del usuario y otra con su lista de recomendaciones

In [11]:
# Rellenar los valores nulos en la columna 'recommend' con False
df_user_item['recommend'] = df_user_item['recommend'].fillna(False)
# Filtrar solo las filas con recommend igual a True
df_user_item = df_user_item[df_user_item['recommend']]

# Agrupar por user_id y agregar una lista de juegos recomendados
df_user_item = df_user_item.groupby('user_id').agg({'item_name': list}).reset_index()


In [23]:
df_user_item

,user_id,item_name
0,--000--,"[ACE - Arena: Cyber Evolution, APB Reloaded, A..."
1,--ace--,"[8BitBoy, AdVenture Capitalist, BattleBlock Th..."
2,--ionex--,"[Another World, Call of Duty: Modern Warfare 2..."
3,-2SV-vuLB-Kg,"[16 Bit Arena, 16bit Trader, A grande bagun√ßa..."
4,-Azsael-,"[Amnesia: A Machine for Pigs, Anno 2070, Arcan..."
...,...,...
20157,zwanzigdrei,"[Always Sometimes Monsters, Archeblade, Awesom..."
20158,zy0705,"[Dota 2 Test, Marvel Heroes 2016, Realm of the..."
20159,zynxgameth,"[Awesomenauts, BattleBlock Theater, Broforce, ..."
20160,zyr0n1c,"[Agarest: Generations of War, America's Army: ..."


### Modelo

In [15]:

# Dividir el conjunto de datos en partes más pequeñas
chunk_size = 10000
num_chunks = len(df_user_item_ML) // chunk_size
chunks = np.array_split(df_user_item_ML, num_chunks)

# Inicializar una lista para almacenar los resultados
similar_users_list = []

# Encontrar el número mínimo de columnas entre todas las matrices
min_columns = float('inf')

# Encontrar y almacenar los 5 usuarios más similares para cada parte
for chunk in chunks:
    user_features = chunk[['items_count', 'playtime_forever']]
    cosine_sim_chunk = cosine_similarity(user_features)

    # Actualizar el número mínimo de columnas
    min_columns = min(min_columns, cosine_sim_chunk.shape[1])
    
    # Encontrar los 5 usuarios más similares para cada usuario en la parte actual
    for i, user_id in enumerate(chunk['user_id']):
        similar_users = chunk['user_id'].iloc[cosine_sim_chunk[i].argsort()[::-1][1:6]].tolist()
        similar_users_list.append({'user_id': user_id, 'similar_users': similar_users})

# Ajustar todas las matrices para tener el mismo número de columnas
adjusted_similar_users_list = [{'user_id': entry['user_id'], 'similar_users': entry['similar_users'][:min_columns]} for entry in similar_users_list]

# Crear un DataFrame a partir de la lista de resultados
result_df = pd.DataFrame(adjusted_similar_users_list)

# Mostrar el DataFrame resultante
print(result_df)


# Dividir el conjunto de datos en partes más pequeñas
chunk_size = 10000
num_chunks = len(df_user_item_ML) // chunk_size
chunks = np.array_split(df_user_item_ML, num_chunks)

# Inicializar una lista para almacenar los resultados
similar_users_list = []

# Encontrar el número mínimo de columnas entre todas las matrices
min_columns = float('inf')

# Encontrar y almacenar los 5 usuarios más similares para cada parte
for chunk in chunks:
    user_features = chunk[['items_count', 'playtime_forever']]
    cosine_sim_chunk = cosine_similarity(user_features)

    # Actualizar el número mínimo de columnas
    min_columns = min(min_columns, cosine_sim_chunk.shape[1])
    
    # Encontrar los 5 usuarios más similares para cada usuario en la parte actual
    for i, user_id in enumerate(chunk['user_id']):
        similar_users = chunk['user_id'].iloc[cosine_sim_chunk[i].argsort()[::-1][1:6]].tolist()
        similar_users_list.append({'user_id': user_id, 'similar_users': similar_users})

# Ajustar todas las matrices para tener el mismo número de columnas
adjusted_similar_users_list = [{'user_id': entry['user_id'], 'similar_users': entry['similar_users'][:min_columns]} for entry in similar_users_list]

# Crear un DataFrame a partir de la lista de resultados
result_df = pd.DataFrame(adjusted_similar_users_list)


                 user_id                                      similar_users
0                --000--  [76561197963538874, 76561198006074188, 7656119...
1                --ace--  [76561197994950912, 76561198037453247, 7656119...
2              --ionex--  [76561197960499156, 2092000666, 76561198038604...
3           -2SV-vuLB-Kg  [76561198043082457, 76561197967856025, 7656119...
4      -404PageNotFound-  [12435gg, 76561198014300427, 76561197969020980...
...                  ...                                                ...
70907             zzonci  [stevieisawesome, killermemestar123, kydmar, m...
70908        zzoptimuszz  [sparid, lynchyknight1, misterwolfshanks, piku...
70909            zzydrax  [leejiajun, jerejosh2, maddydufall, secretlego...
70910              zzyfo  [kennygetsu, l3wis992, ladysaphera, william079...
70911         zzzmidmiss  [sbotfield0211, rawritspeter, samzero, jaice, ...

[70912 rows x 2 columns]
                 user_id                                      

In [21]:
# Realizar left join
merged_df = pd.merge(result_df, df_user_item, how='left', left_on='user_id', right_on='user_id')

# Agrupar por 'user_id' y crear una lista de juegos recomendados
result_df_final = merged_df.groupby('user_id')['item_name'].apply(list).reset_index()

# Renombrar las columnas
result_df_final.columns = ['user_id', 'recomendaciones']


In [66]:
## Explode la columna similar_users para tener un usuario similar por fila
user_item = result_df.explode('similar_users')
#Concatenamos similar_users con df_user_item[user_id]
# Realizar left join
user_item = pd.merge(user_item, df_user_item, how='left', left_on='similar_users', right_on='user_id')
#Filtramos los nan
user_item.dropna(inplace=True)
#Seleccionamos las columnas que nos interesa
# Seleccionar las columnas relevantes
user_item = user_item[['user_id_x', 'item_name']]
#Renombramos las columnas
user_item.columns = ['user_id', 'recomendaciones']
#Como hay mas de un usuario similar que hizo recomendaciones , hay varias filas por user_id , agrupamos
user_item = user_item.groupby('user_id')['recomendaciones'].sum().reset_index()
#Filtramos los primeros elementos de la lista de recomendaciones
user_item['recomendaciones'] = user_item['recomendaciones'].apply(lambda x: x[:5])


In [67]:
user_item

,user_id,recomendaciones
0,--000--,"[ARK: Survival Evolved, ARK: Survival Of The F..."
1,--ace--,"[Alan Wake, Alan Wake's American Nightmare, As..."
2,--ionex--,"[A.V.A - Alliance of Valiant Arms, Afterfall I..."
3,-2SV-vuLB-Kg,"[Age of Empires II: HD Edition, Age of Empires..."
4,-404PageNotFound-,"[3DMark, 7 Days to Die, Age of Empires II: HD ..."
...,...,...
51914,zzonci,[ACE COMBAT‚Ñ¢ ASSAULT HORIZON Enhanced Editio...
51915,zzoptimuszz,"[7 Days to Die, Age of Empires II: HD Edition,..."
51916,zzydrax,"[ARK: Survival Evolved, ARK: Survival Of The F..."
51917,zzyfo,"[Ace of Spades, Anomaly Warzone Earth, Aquaria..."


In [68]:
user_item.to_csv('..\\Datsets\\ModeloML\\recomendacion_user_item.csv', index=False)

In [69]:

def recomendacion_user_item(user_id):
    """
    Retorna la lista de recomendaciones para un usuario específico.

    Parameters:
    - user_item: DataFrame que contiene las columnas user_id y recomendaciones.
    - user_id: El usuario para el cual se desea obtener las recomendaciones.

    Returns:
    - Lista de recomendaciones para el usuario especificado.
    """
    # Filtra el DataFrame para obtener solo las filas correspondientes al usuario dado
    usuario_filtro = user_item[user_item['user_id'] == user_id]

    # Verifica si el usuario existe en el DataFrame
    if usuario_filtro.empty:
        return f"No se encontraron recomendaciones para el usuario {user_id}"

    # Concatena las listas de recomendaciones correspondientes al usuario
    recomendaciones_usuario = usuario_filtro['recomendaciones'].sum()[0:5]

    return recomendaciones_usuario




In [70]:
# Ejemplo de uso:
usuario_deseado = '--000--'
recomendaciones_para_usuario = recomendacion_user_item (usuario_deseado)
print(f"Recomendaciones para el usuario {usuario_deseado}:\n{recomendaciones_para_usuario}")

Recomendaciones para el usuario --000--:
['ARK: Survival Evolved', 'ARK: Survival Of The Fittest', 'Amnesia: The Dark Descent', 'Archeblade', 'Blacklight: Retribution']
